In [3]:
%pip install pydantic-ai

  Using cached pydantic_ai-0.0.49-py3-none-any.whl.metadata (11 kB)
  Using cached pydantic_ai_slim-0.0.49-py3-none-any.whl.metadata (3.6 kB)
  Using cached anthropic-0.49.0-py3-none-any.whl.metadata (24 kB)
  Using cached boto3-1.37.26-py3-none-any.whl.metadata (6.7 kB)
  Using cached cohere-5.14.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached groq-0.22.0-py3-none-any.whl.metadata (15 kB)
  Using cached mcp-1.6.0-py3-none-any.whl.metadata (20 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
Using cached pydantic_ai-0.0.49-py3-none-any.whl (10 kB)
Using cached pydantic_ai_slim-0.0.49-py3-none-any.whl (142 kB)
Using cached anthropic-0.49.0-py3-none-any.whl (243 kB)
Using cached boto3-1.37.26-py3-none-any.whl (139 kB)
Using cached cohere-5.14.2-py3-none-any.whl (259 kB)
Using cached google_auth-2.38.0-py2.py3-none-any.whl (210 kB)
Using cached groq-0.22.0-py3-none-any.whl (126 kB)
Using cached mcp-1.6.0-py3-none-any.whl (76 kB)
Note: you may need to restar

In [4]:
from pydantic_ai.models.openai import OpenAIModel

help(OpenAIModel)

Help on class OpenAIModel in module pydantic_ai.models.openai:

class OpenAIModel(pydantic_ai.models.Model)
 |  OpenAIModel(model_name: 'OpenAIModelName', *, provider: "Literal['openai', 'deepseek', 'azure'] | Provider[AsyncOpenAI]" = 'openai', system_prompt_role: 'OpenAISystemPromptRole | None' = None)
 |  
 |  A model that uses the OpenAI API.
 |  
 |  Internally, this uses the [OpenAI Python client](https://github.com/openai/openai-python) to interact with the API.
 |  
 |  Apart from `__init__`, all methods are private or match those of the base class.
 |  
 |  Method resolution order:
 |      OpenAIModel
 |      pydantic_ai.models.Model
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, model_name: 'OpenAIModelName', *, provider: "Literal['openai', 'deepseek', 'azure'] | Provider[AsyncOpenAI]" = 'openai', system_prompt_role: 'OpenAISystemPromptRole | None' = None)
 |      Initializ